In [1]:
import requests
import pandas as pd
from time import sleep
import numpy as np
from tqdm import tqdm
import os
import pickle
import re
from bs4 import BeautifulSoup

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import numpy as np
from IPython.display import clear_output

import warnings
warnings.filterwarnings('ignore')

# Importing pickle file

In [2]:
##cambiar a artist
with open('../data/pickle/artist.pickle', 'rb') as tracks:
    tracks = pickle.load(tracks)

In [3]:
tracks.head()

,url,artist_clean,track_clean
0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


In [4]:
tracks.rename(columns={'track_clean':'track','artist_clean':'artist'}, inplace=True)
tracks.reset_index(inplace=True)
tracks.head()

,index,url,artist,track
0,0,https://open.spotify.com/track/4Km5HrUvYTaSUfi...,migos,bad and boujee (feat. lil uzi vert)
1,1,https://open.spotify.com/track/343YBumqHu19cGo...,drake,fake love
2,2,https://open.spotify.com/track/5aAx2yezTd8zXrk...,"the weeknd, daft punk",starboy
3,3,https://open.spotify.com/track/7BKLCZ1jbUBVqRi...,"the chainsmokers, halsey",closer
4,4,https://open.spotify.com/track/6fujklziTHa8uoM...,"rae sremmurd, gucci mane",black beatles


# Beautiful soup + selenium

To get the info, this script combines selenium to access and search for the different songs and beautifulSoup to get the text info from the tracks.

In [ ]:
almacen_paginas = {
    'pagina' : [], 
    'lyrics':[],
    'featuring' : [],
    'produced_by': [],
    'label': [],
    'written by': [],
    'drums': [],
    'guitar': [],
    'recorded_at' : [],
    'release_date':[],
    'url':[]
}

In [5]:
with open('../data/pickle/genius_lyrics.pickle', 'rb') as almacen_paginas:
    almacen_paginas = pickle.load(almacen_paginas)

In [136]:
#test = tracks.iloc[(864):,:].reset_index(drop=True)
test = tracks.iloc[(len(almacen_paginas['url'])):,:].reset_index(drop=True)
filename = 'genius_lyrics'
print(test.shape)
test.head()

(2604, 4)


,index,url,artist,track
0,381586,https://open.spotify.com/track/5Joh6MZQpsL4fww...,mohandbadz,samany setk
1,381609,https://open.spotify.com/track/5irzewJoHob42Nq...,russ,seduce (feat. capella grey)
2,382108,https://open.spotify.com/track/169koB619bHoCZG...,"clueless kit, køra",come to a stop (feat. kora)
3,382115,https://open.spotify.com/track/41VdpwFT6QK2mB8...,sam seg,lover
4,382125,https://open.spotify.com/track/3EJaUuGn4gtGa7u...,overcoats,cherry wine


In [135]:
#almacen_paginas['pagina'].append(np.nan)
#almacen_paginas['lyrics'].append(np.nan)
#almacen_paginas['url'].pop()
print(len(almacen_paginas['lyrics']), len(almacen_paginas['pagina']), len(almacen_paginas['url']))

12055 12055 12055


In [137]:
## iniciamos el web driver
opciones= Options()
opciones.add_argument("--start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get('https://genius.com/')
driver.maximize_window()
driver.set_window_size(1920, 1080)
## te metes dentro de la página y con inspeccionar, seleccionas el elemento de las cookies --> copy / selector
##por si tu ordenador es un poco lento, le puedes poner un tiempo para que intente hacerlo
driver.implicitly_wait(30)
for index, row in test.iterrows():
    print(index, row['artist'].split(',')[0],'---', row['track'].split('-')[0])
    almacen_paginas['url'].append(row['url'])
    if index == 0: 
        driver.find_element('css selector', '#onetrust-accept-btn-handler').click()
        #sleep(1)
        ## busco el nombre de canción que quiero buscar
        driver.find_element('css selector', '#application > div > div.PageHeaderdesktop__Container-bhx5ui-0.dmNhEr > div.PageHeaderdesktop__Left-bhx5ui-2.ctcvqz > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'].split('-')[0], Keys.ENTER)
        #sleep(1)
    else:
        ## busco el nombre de canción que quiero buscar
        driver.find_element('css selector', '#sticky-nav > div.StickyNavdesktop__Right-sc-60o25r-2.kMDkxm > form > input').send_keys(row['artist'].split(',')[0] + ' ' + row['track'].split('-')[0], Keys.ENTER)
       # sleep(1)
    ## selecciono el resultado de la búsqueda que quiero
    try:
        driver.find_element('xpath', '/html/body/routable-page/ng-outlet/search-results-page/div/div[2]/div[1]/div[2]/search-result-section/div/div[2]/search-result-items/div[1]/search-result-item/div/mini-song-card/a').click()
        driver.implicitly_wait(30)
        ## clico el all credits
        try:
            driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[4]/div/a').click()
        except:
            try:
                driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[3]/div/a').click()
            except:
                try:
                    driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[2]/div/a').click()
                except:
                    try:
                        driver.find_element('xpath', '//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div[1]/div/a').click()
                    except:
                        print('oye que no va')
                        break
        #'//*[@id="application"]/main/div[1]/div[4]/div/div[1]/div/div/a'
        #sleep(1)
        page_source = driver.page_source
        url = driver.current_url
        res = requests.get(url + '#song-info')
        soup = BeautifulSoup(res.content, 'html.parser')
        pagina = soup.text
        almacen_paginas['pagina'].append(pagina)
        try:
            almacen_paginas['lyrics'].append(re.findall(r'Lyrics(.*)EmbedCancel?', pagina)[0])
        except:
            almacen_paginas['lyrics'].append(np.nan)
        with open(f'../data/pickle/{filename}.pickle', 'wb') as data_scrapeado:
            pickle.dump(almacen_paginas, data_scrapeado)
        clear_output(wait=True)
    except:
        almacen_paginas['pagina'].append(np.nan)
        almacen_paginas['lyrics'].append(np.nan)
        clear_output(wait=True)

1 russ --- seduce (feat. capella grey)


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#sticky-nav > div.StickyNavdesktop__Right-sc-60o25r-2.kMDkxm > form > input"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00636643]
	(No symbol) [0x005CBE21]
	(No symbol) [0x004CDA9D]
	(No symbol) [0x00501342]
	(No symbol) [0x0050147B]
	(No symbol) [0x00538DC2]
	(No symbol) [0x0051FDC4]
	(No symbol) [0x00536B09]
	(No symbol) [0x0051FB76]
	(No symbol) [0x004F49C1]
	(No symbol) [0x004F5E5D]
	GetHandleVerifier [0x008AA142+2497106]
	GetHandleVerifier [0x008D85D3+2686691]
	GetHandleVerifier [0x008DBB9C+2700460]
	GetHandleVerifier [0x006E3B10+635936]
	(No symbol) [0x005D4A1F]
	(No symbol) [0x005DA418]
	(No symbol) [0x005DA505]
	(No symbol) [0x005E508B]
	BaseThreadInitThunk [0x772B7D69+25]
	RtlInitializeExceptionChain [0x77DDBB9B+107]
	RtlClearBits [0x77DDBB1F+191]


# Export

In [ ]:
with open('../data/pickle/genius_lyrics.pickle', 'wb') as data_genius:
    pickle.dump(tracks, data_genius)